In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
torch.manual_seed(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p2/X_test.csv
/kaggle/input/2021-ai-w6-p2/train.csv
/kaggle/input/2021-ai-w6-p2/sample_submit.csv


In [2]:
X = pd.read_csv("/kaggle/input/2021-ai-w6-p2/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w6-p2/X_test.csv").drop(["ID", "Day"], axis = 1)
y = X["TARGET"]
X = X.drop(["Day", "TARGET"], axis = 1)

In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
test = sc.transform(test)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
X = torch.FloatTensor(X).to(device)
test = torch.FloatTensor(test).to(device)
y = torch.FloatTensor(y.to_numpy()).reshape(-1, 1).to(device)

In [6]:
y.shape

torch.Size([9999, 1])

In [7]:
linear1 = torch.nn.Linear(7, 10, bias = True).to(device)
linear2 = torch.nn.Linear(10, 10, bias = True).to(device)
linear3 = torch.nn.Linear(10, 10, bias = True).to(device)
linear4 = torch.nn.Linear(10, 1, bias = True).to(device)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid,linear2, sigmoid,linear3,sigmoid, linear4)


In [8]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-2)

for stop in range(30001):
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = loss(hypothesis, y)
    cost.backward()
    optimizer.step()
    
    if stop%1000==0:
        print(stop, cost.item())

0 1022.8781127929688
1000 9.235820770263672
2000 4.677814483642578
3000 3.4402990341186523
4000 2.7870142459869385
5000 2.352090835571289
6000 2.042922019958496
7000 1.8294864892959595
8000 1.6859264373779297
9000 1.5861531496047974
10000 1.5123088359832764
11000 1.4546889066696167
12000 1.4082728624343872
13000 1.3702293634414673
14000 1.3387694358825684
15000 1.3124724626541138
16000 1.2900155782699585
17000 1.270470142364502
18000 1.2532552480697632
19000 1.2379740476608276
20000 1.2244908809661865
21000 1.212693691253662
22000 1.2026435136795044
23000 1.1944491863250732
24000 1.188151240348816
25000 1.1833728551864624
26000 1.1794371604919434
27000 1.1756680011749268
28000 1.171379566192627
29000 1.166346788406372
30000 1.1605019569396973


In [9]:
predict = model(test)

In [10]:
predict = predict.detach().numpy()

In [11]:
submit = pd.read_csv("/kaggle/input/2021-ai-w6-p2/sample_submit.csv")
submit["TARGET"] = predict

In [12]:
submit.head(20)

,ID,TARGET
0,0,10.269402
1,1,18.880264
2,2,27.009249
3,3,34.423557
4,4,40.915970
5,5,46.971523
6,6,45.411144
7,7,55.787849
8,8,50.159004
9,9,49.435860


In [13]:
submit.to_csv("submission.csv", index=False)